Notebook which will:
- Obtain alert photometry for named SN and convert to AMPEL LightCurve object.
- Match with catalogs from the DESY match service.
- Carry out a model fit using T2Parsnip

In [ ]:
import pandas as pd
import requests, os, time
from requests.auth import HTTPBasicAuth
import numpy as np
import matplotlib.pyplot as plt
#import lcdata
from astropy.table import Table
from IPython.core.display import Image, display

In [ ]:
import ampel_quick_import

In [ ]:
%qi AmpelLogger ZTFIdMapper ZiDataPointShaper T1Document LightCurve T2RunParsnip

In [ ]:
snname = "ZTF20aaumsrr"
token = os.environ["ARCHIVE_TOKEN"]
logger = AmpelLogger.get_logger()

In [ ]:
def api_to_lightcurve(name, token, shaper=None):
    """
    Retrieve the alert history of a SN and convert to LightCurve
    
    """
    
    # Setup connection
    endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/v3/object/{}/photopoints".format(name)
    header = {"Authorization": "bearer "+token}

    response = requests.get(endpoint, headers=header )
    
    if not response.ok:
        print('... failed to get alert')
        return None
    
    # Convert
    alert = response.json()
    if alert is None:
        print(' ... no alert content')
        return None
    
    pps = [alert['candidate']]
    pps.extend( [prv_cand for prv_cand in alert['prv_candidates'] ] )

    if shaper is None:
        tmplog = AmpelLogger.get_logger()
        shaper = ZiDataPointShaper(logger=tmplog)
    
    stockId = ZTFIdMapper.to_ampel_id(name)
    dps = shaper.process( pps, stockId)
    t1d = T1Document(stock=stockId, link=0)
    return LightCurve.build(t1d, dps)

In [ ]:
lc = api_to_lightcurve(snname, token=token)

I. Run Parsnip using a single fixed redshift. Note that in live processing redshifts are  dynamically provied through chaining to the DigestRedshift unit. See T2RunSncosmo for demo on how to first search catalogs. 

In [ ]:
parsnip_config = {
    'parsnip_model':'/home/jnordin/data/parsnip/parsnip_ps1_ztf.pt', 
    'parsnip_classifier':'/home/jnordin/data/parsnip/ztf_classifier.pkl', 
    'fixed_z' : 0.1,
    't2_dependency': [],
    'plot_suffix': 'png',
    'plot_dir': '/home/jnordin/tmp',
    'apply_mwcorrection': True,
}

In [ ]:
t2parsnip = T2RunParsnip( **parsnip_config, logger=logger)

In [ ]:
t2parsnip.post_init()

In [ ]:
parsnip_out = t2parsnip.process(lc, t2_views=[])

In [ ]:
parsnip_out

In [ ]:
fname = "{}/t2parsnip_{}.{}".format(parsnip_config['plot_dir'],snname,parsnip_config['plot_suffix']) 
print(fname)

In [ ]:
display(Image(filename=fname))

II. Run Parsnip using a list of fixed redshifts. The individual results will be found in the output, together with marginalized results as well as the one for the min chi2/dof. 

In [ ]:
parsnip_config = {
    'parsnip_model':'/home/jnordin/data/parsnip/parsnip_ps1_ztf.pt', 
    'parsnip_classifier':'/home/jnordin/data/parsnip/ztf_classifier.pkl', 
    'fixed_z' : [0.03, 0.07, 0.1,0.15, 0.2,0.3],
    't2_dependency': [],
    'plot_suffix': 'png',
    'plot_dir': '/home/jnordin/tmp',
    'apply_mwcorrection': True,
}

In [ ]:
t2parsnip = T2RunParsnip( **parsnip_config, logger=logger)

In [ ]:
t2parsnip.post_init()

In [ ]:
parsnip_out = t2parsnip.process(lc, t2_views=[]) 

In [ ]:
display(Image(filename=fname))

In [ ]:
parsnip_out